In [1]:
import cv2
from tracker import *
import torch
import numpy as np

## 1. Load YOLO Object Detector

In [3]:
# Cargar el modelo YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Definir clases de interés
CLASSES_OF_INTEREST = ['car']

Using cache found in C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-22 Python-3.9.7 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## 2. Define Tracker Class (per centroides)

In [48]:
import numpy as np
from scipy.spatial.distance import cdist

class Tracker:
    MAX_DISAPPEAR_LIMIT = 3
    def __init__(self):
        self.next_unique_id = 0
        self.trackers = {}
        self.disappear_trackers = {}
        self.tracked_bboxes = {}
    
    
    def init_object(self,centroid,boxes):
        global next_unique_id
        self.trackers[self.next_unique_id] = centroid
        self.tracked_bboxes[self.next_unique_id] = boxes
        self.disappear_trackers[self.next_unique_id] = 0
        self.next_unique_id+=1

    def del_object(self,track_id):
        del self.trackers[track_id]
        del self.tracked_bboxes[track_id]
        del self.disappear_trackers[track_id]

    def update_object(self,bboxes):
        
        if(len(bboxes)==0):
            
            for oid in list(self.disappear_trackers.keys()):
                self.disappear_trackers[oid]+=1
                
                if self.disappear_trackers[oid] > Tracker.MAX_DISAPPEAR_LIMIT:
                    self.del_object(oid)
                
            return self.tracked_bboxes
        
        else:   
            input_centroids = np.zeros((len(bboxes),2)) 
            for i in range(len(bboxes)):
                x,y,w,h = bboxes[i][0],bboxes[i][1],bboxes[i][2],bboxes[i][3]
                cx,cy = x + w/2 , y + h/2
                input_centroids[i] = (cx,cy)

            
            if(len(self.trackers)==0):
                for i in range(len(input_centroids)):
                    self.init_object(input_centroids[i],bboxes[i])
            
            else:
                
                tracker_centroids = list(self.trackers.values())

                distance_matrix = cdist(np.array(tracker_centroids) , input_centroids)

                rows = distance_matrix.min(axis=1).argsort()
                cols = distance_matrix.argmin(axis=1)[rows]

                usedRows = set()
                usedCols = set()
                
                tracker_ids = list(self.trackers.keys()) 
                for row,col in zip(rows,cols):
                    if row in usedRows or col in usedCols:
                        continue
                    track_id = tracker_ids[row]
                    
                    self.trackers[track_id] = input_centroids[col]
                    self.tracked_bboxes[track_id] = bboxes[col]

                    self.disappear_trackers[track_id] = 0
                    usedRows.add(row)                                
                    usedCols.add(col)
                unusedRows = set(range(0,distance_matrix.shape[0])).difference(usedRows)
                unusedCols = set(range(0,distance_matrix.shape[1])).difference(usedCols)
                if(distance_matrix.shape[0]>=distance_matrix.shape[1]):
                    
                    for r in unusedRows: 
                        track_id = tracker_ids[r]
                        self.disappear_trackers[track_id]+=1
                        if(self.disappear_trackers[track_id] > Tracker.MAX_DISAPPEAR_LIMIT):
                            self.del_object(track_id)
                else:
                    for c in unusedCols:                    
                        self.init_object(input_centroids[c],bboxes[c])

        return self.tracked_bboxes

## 3. Load Video

In [54]:
# Create tracker object
# tracker = EuclideanDistTracker()
tracker2 = Tracker()

cap = cv2.VideoCapture("/Users/mirvi/Desktop/mii/UAB/4.1/PSIV2/detect mateicules/repte2_psiv2/data_r2/short_uab_flow.mp4")

## 4. Object Detection and Tracking over video

In [56]:
# Loop over video frames
while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    roi = frame[350: 820,100:520]

    # 1. Object Detection
    results = model(roi)

    detections = []
    for det in results.xyxy[0]:
        x1, y1, x2, y2, conf, cls = det

        if model.names[int(cls)] == 'car':
            w = int(x2 - x1)
            h = int(y2 - y1)
            x = int(x1)
            y = int(y1)
            detections.append([x, y, w, h])

    # 2. Object Tracking
    # Update tracker
    boxes_ids = tracker2.update_object(detections)
    # boxes_ids = tracker.update(detections)
    for box_id,box in boxes_ids.items():
        x, y, w, h = box
        id = box_id
        cv2.putText(roi, str(id), (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 0.7, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)

    # 3. Control steps de frame
    cap.grab()

    key = cv2.waitKey(30)
    # escape key
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


0 [327, 144, 90, 86]
1 [308, 97, 94, 88]


C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


0 [327, 144, 90, 87]
1 [308, 97, 94, 87]


C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

0 [325, 143, 92, 87]
1 [306, 97, 97, 88]
2 [178, 80, 95, 85]


C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


0 [325, 143, 92, 86]
1 [307, 97, 96, 87]
2 [180, 77, 93, 85]


C:\Users\mirvi/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


0 [325, 143, 92, 87]
1 [307, 97, 96, 87]
2 [180, 74, 93, 84]
